<a href="https://colab.research.google.com/github/anne030303/data-course-sample/blob/main/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [ ]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-24 16:32:26--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.1’

All_Beauty.csv.1    100%[===================>]  14.78M  18.6MB/s    in 0.8s    

2021-12-24 16:32:27 (18.6 MB/s) - ‘All_Beauty.csv.1’ saved [15499476/15499476]

--2021-12-24 16:32:27--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.1’

meta_All_Beauty.jso 100%[===================>]   9.85M  14.0MB/s    in 0.7s   

In [ ]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
metadata_new = metadata[['asin', 'brand', 'title', 'price', 'rank', 'description', 'also_view', 'also_buy']].copy()
metadata_new.loc[:,'rank_num'] = metadata_new['rank'].str.split('in', n = 1, expand = True)[0].str.replace(',','')
metadata_new.loc[:,'rank_cat'] = metadata_new['rank'].str.split('in', n = 1, expand = True)[1].str.replace('(','')
metadata_new.loc[:,'price'] = pd.to_numeric(metadata_new['price'].str.replace('$',''), errors='coerce')
metadata_new.head()

,asin,brand,title,price,rank,description,also_view,also_buy,rank_num,rank_cat
0,6546546450,idea village,Loud 'N Clear&trade; Personal Sound Amplifier,NaN,"2,938,573 in Beauty & Personal Care (",[Loud 'N Clear Personal Sound Amplifier allows...,[],[],2938573,Beauty & Personal Care
1,7178680776,,No7 Lift &amp; Luminate Triple Action Serum 50...,44.99,"872,854 in Beauty & Personal Care (",[No7 Lift & Luminate Triple Action Serum 50ml ...,[],"[B01E7LCSL6, B008X5RVME]",872854,Beauty & Personal Care
2,7250468162,No7,No7 Stay Perfect Foundation Cool Vanilla by No7,28.76,"956,696 in Beauty & Personal Care (",[No7 Stay Perfect Foundation now stays perfect...,"[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]",[],956696,Beauty & Personal Care
3,7367905066,,Wella Koleston Perfect Hair Colour 44/44 Mediu...,NaN,"1,870,258 in Beauty & Personal Care (",[],[],[B0041PBXX8],1870258,Beauty & Personal Care
4,7414204790,Pirmal Healthcare,Lacto Calamine Skin Balance Oil control 120 ml...,12.15,"67,701 in Beauty & Personal Care (",[Lacto Calamine Skin Balance Daily Nourishing ...,"[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...",[],67701,Beauty & Personal Care


In [ ]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]

ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [ ]:
ratings_by_asin = ratings_trainings.groupby('asin').mean()['overall'].reset_index()[['asin', 'overall']]#.set_index('asin')
ratings_sku_detail = pd.merge(ratings_by_asin,metadata_new)
ratings_sku_detail.head()

,asin,overall,brand,title,price,rank,description,also_view,also_buy,rank_num,rank_cat
0,6546546450,2.5,idea village,Loud 'N Clear&trade; Personal Sound Amplifier,NaN,"2,938,573 in Beauty & Personal Care (",[Loud 'N Clear Personal Sound Amplifier allows...,[],[],2938573,Beauty & Personal Care
1,7178680776,3.0,,No7 Lift &amp; Luminate Triple Action Serum 50...,44.99,"872,854 in Beauty & Personal Care (",[No7 Lift & Luminate Triple Action Serum 50ml ...,[],"[B01E7LCSL6, B008X5RVME]",872854,Beauty & Personal Care
2,7250468162,5.0,No7,No7 Stay Perfect Foundation Cool Vanilla by No7,28.76,"956,696 in Beauty & Personal Care (",[No7 Stay Perfect Foundation now stays perfect...,"[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]",[],956696,Beauty & Personal Care
3,7367905066,5.0,,Wella Koleston Perfect Hair Colour 44/44 Mediu...,NaN,"1,870,258 in Beauty & Personal Care (",[],[],[B0041PBXX8],1870258,Beauty & Personal Care
4,7414204790,4.4,Pirmal Healthcare,Lacto Calamine Skin Balance Oil control 120 ml...,12.15,"67,701 in Beauty & Personal Care (",[Lacto Calamine Skin Balance Daily Nourishing ...,"[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...",[],67701,Beauty & Personal Care


In [ ]:
def process(data,user,k):
  cat = []
  brand = []
  if data.loc[data['reviewerID']==user].empty == False:
    for item in list(data.loc[data['reviewerID']==user]['asin']):
      #print(item)
      cat.append(ratings_sku_detail.loc[ratings_sku_detail['asin']==item]['rank_cat'].values[0])
      brand.append(ratings_sku_detail.loc[ratings_sku_detail['asin']==item]['brand'].values[0])
    print(cat,brand)
    return False
  return True
  
  #print(data)
  return data[:k]
count = 0
for user in users:
  if process(ratings_trainings,user,10) == False:
    count += 1
  if count == 5:
    break

[' Beauty & Personal Care '] ['']
[' Beauty & Personal Care '] ['kingfansion']
[' Beauty & Personal Care ', ' Beauty & Personal Care '] ['Elite99', 'Elite99']
[' Beauty & Personal Care '] ['']
[' Beauty & Personal Care '] ['Hittime']


## 產生推薦

In [ ]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    Your Code
    '''
    ratings_trainings = training_data
    recommendations = {user: ratings_trainings['asin'].sample(n=k).tolist() for user in users}
    return recommendations


ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

{'A100XQFWKQ30O2': ['B01DKQAXC0',
  'B01CNLBQ0Q',
  'B00U1WGBFA',
  'B00X3T6FEU',
  'B01DL8XFNG',
  'B00MSKXPWU',
  'B000GLRREU',
  'B011B7OXV2',
  'B01CW0URXA',
  '1620213982'],
 'A103T1QOGFCSEH': ['B002C5VTLA',
  'B0167K16H2',
  'B00028LYO6',
  'B00M0UD2IU',
  'B001E5PLCM',
  'B01H3IVSAC',
  'B00NM5A4A6',
  'B00TA30HK6',
  'B00YMLWSDU',
  'B00X4DKZKU'],
 'A106UKKSJ2KXPF': ['B0195R1FT8',
  'B000VV1YOY',
  'B01DKI7B46',
  'B000NJJFBI',
  'B00B7V273E',
  'B000FOI48G',
  'B0007VU9O4',
  'B00MGK9ZNA',
  'B001LFF31Y',
  'B000GLRREU'],
 'A10A7GV4D5A11V': ['B0067F28ZW',
  'B001FB5H9C',
  'B01DKI8S1Q',
  'B00BMVV3MK',
  'B00TUAVLSG',
  'B0054Y1YBQ',
  'B00J7MYH38',
  'B006LTTBMW',
  'B00Q794RMA',
  'B01DKQAXC0'],
 'A1119JJ37ZLB8R': ['B001OHV1H4',
  'B000GLRREU',
  'B00YP2QR38',
  'B006L4PIO2',
  'B001DKQ47E',
  'B004EMKRE4',
  'B002791OAC',
  'B00SQ3S6TA',
  'B000GLRREU',
  'B000X2FPXC'],
 'A113UOOLBSZN52': ['B00TT1D7BK',
  'B0016JDQP0',
  'B001MW1MK2',
  'B000X8H61A',
  'B000NCW562',
  'B00R

## 結果評估

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.010169491525423728